In [58]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

import pandas as pd
from learn.utils import FileHandler, get_file_suffix_map

In [59]:
epsilon_df = pd.read_csv('../experiments/epsilon_greedy_run_0/learning_out/intersection_0/action_hist.csv')
thompson_df = pd.read_csv('../experiments/thompson_run_0/learning_out/intersection_0/action_hist.csv')
ucb_df = pd.read_csv('../experiments/upper_confidence_bound_run_0/learning_out/intersection_0/action_hist.csv')

In [60]:
epsilon_target = epsilon_df[1:][['avg_lane_queueing_length', 'avg_lane_queueing_time']]
thompson_target = thompson_df[1:][['avg_lane_queueing_length', 'avg_lane_queueing_time']]
ucb_target = ucb_df[1:][['avg_lane_queueing_length', 'avg_lane_queueing_time']]

In [61]:
def moving_average_col_generation(target, length):
    target['roll_lane_length'] = target['avg_lane_queueing_length'].rolling(length).mean()
    target['roll_waiting_time'] = target['avg_lane_queueing_time'].rolling(length).mean()
    return target

In [62]:
import plotly.graph_objects as go
def plot_df(df, title=None):
    fig = go.Figure()
    fig.update_layout(title_text=title, title_x=0.5)
    fig.add_trace(go.Scatter(x=df.index, y=df['avg_lane_queueing_length'], name='queue_length', opacity=0.1))
    fig.add_trace(go.Scatter(x=df.index, y=df['avg_lane_queueing_time'], name = 'queue_time', opacity=0.1))
    fig.add_trace(go.Scatter(x=df.index, y=df['roll_lane_length'], name='moving_avg_queue_length'))
    fig.add_trace(go.Scatter(x=df.index, y=df['roll_waiting_time'], name = 'moving_avg_queue_time'))
    fig.show()

In [63]:
def vis(raw_df, moving_avg_length, title=None):
    df = moving_average_col_generation(raw_df, moving_avg_length)
    plot_df(df, title)
    percentage_length = float(df['roll_lane_length'][moving_avg_length] - df['roll_lane_length'][len(df)-1]) / df['roll_lane_length'][moving_avg_length]
    percentage_time = float(df['roll_waiting_time'][moving_avg_length] - df['roll_waiting_time'][len(df)-1]) / df['roll_waiting_time'][moving_avg_length]
    print(f'Waiting lane length reduced: {round(percentage_length,2) * 100}%')
    print(f'Waiting time reduced: {round(percentage_time,2) * 100}%')

In [64]:
vis(thompson_target, 100, 'Thompson Sampling Result (100 Moving Average)')

Waiting lane length reduced: 11.0%
Waiting time reduced: 17.0%


In [65]:
vis(epsilon_target, 100, 'Epsilon Greedy Result (100 Moving Average)')

Waiting lane length reduced: 19.0%
Waiting time reduced: 8.0%


This approach is not robust and stable becase each choice of the idividual light might affect each other

In [66]:
vis(ucb_df, 100, 'Upper Confidence Bound Result (100 Moving Average)')

Waiting lane length reduced: 0.0%
Waiting time reduced: 2.0%


In [ ]:
epsilon_df = pd.read_csv('../experiments/epsilon_greedy_compose_run_0/learning_out/intersection_compose/action_hist.csv')
thompson_df = pd.read_csv('../experiments/thompson_run_compose_0/learning_out/intersection_compose/action_hist.csv')
ucb_df = pd.read_csv('../experiments/upper_confidence_bound_compose_run_0/learning_out/intersection_compose/action_hist.csv')

epsilon_target = epsilon_df[1:][['avg_lane_queueing_length', 'avg_lane_queueing_time']]
thompson_target = thompson_df[1:][['avg_lane_queueing_length', 'avg_lane_queueing_time']]
ucb_target = ucb_df[1:][['avg_lane_queueing_length', 'avg_lane_queueing_time']]

In [ ]:
vis(thompson_target, 100, 'Thompson Sampling Result (100 Moving Average)')

In [ ]:
vis(epsilon_target, 100, 'Epsilon Greedy Result (100 Moving Average)')

In [ ]:
vis(ucb_df, 50, 'Upper Confidence Bound Result (50 Moving Average)')